### Text Classification With Machine Learning and SpaCy
+ Text categorization / text classification is the task of assigning predefined categories to documents.
+ Sentiment Analysis
+ Multilabel classification
+ + DataSet source http://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences

##### Aim is to classify reviews into positive or negative review


In [695]:
# Load EDA packages
import pandas as pd

In [696]:
# Load our dataset
df_yelp = pd.read_table('yelp_labelled.txt')
df_imdb = pd.read_table('imdb_labelled.txt')
df_amz = pd.read_table('amazon_cells_labelled.txt')

In [697]:
# Concatenate our Datasets
frames = [df_yelp,df_imdb,df_amz]

In [698]:
# Renaming Column Headers
for colname in frames:
    colname.columns = ["Message","Target"]

In [699]:
# Column names
for colname in frames:
    print(colname.columns)

Index(['Message', 'Target'], dtype='object')
Index(['Message', 'Target'], dtype='object')
Index(['Message', 'Target'], dtype='object')


In [700]:
# Assign a Key to Make it Easier
keys = ['Yelp','IMDB','Amazon']

In [701]:
# Merge or Concat our Datasets
df = pd.concat(frames,keys=keys)

In [702]:
# Length and Shape 
df.shape

(2745, 2)

In [703]:
df.head()

Message  Target
Yelp 0                                 Crust is not good.       0
     1          Not tasty and the texture was just nasty.       0
     2  Stopped by during the late May bank holiday of...       1
     3  The selection on the menu was great and so wer...       1
     4     Now I am getting angry and I want my damn pho.       0

In [704]:
df.to_csv("sentimentdataset.csv")

In [705]:
# Data Cleaning
df.columns

Index(['Message', 'Target'], dtype='object')

In [706]:
# Checking for Missing Values
df.isnull().sum()

Message    0
Target     0
dtype: int64

###  Working with SpaCy
+ Removing Stopwords
+ Lemmatizing

In [707]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_trf')

In [708]:
# Build a list of stopwords to use to filter
stopwords = list(STOP_WORDS)

In [709]:
stopwords

['whereas',
 'most',
 'even',
 'among',
 'eleven',
 'get',
 'further',
 'beyond',
 'whoever',
 'may',
 'no',
 'side',
 'due',
 'across',
 'up',
 'why',
 'down',
 'nor',
 'already',
 'name',
 'will',
 'themselves',
 'of',
 'also',
 'seems',
 'bottom',
 'three',
 'hundred',
 'what',
 'myself',
 'beside',
 'together',
 'about',
 'latter',
 'mostly',
 'anyhow',
 'more',
 'after',
 'except',
 'part',
 'while',
 'make',
 'or',
 'should',
 'indeed',
 'whereby',
 'mine',
 'formerly',
 'two',
 'above',
 'around',
 'nothing',
 '’s',
 'toward',
 'whenever',
 'perhaps',
 'moreover',
 'have',
 '‘ll',
 'since',
 'seemed',
 'see',
 'with',
 'amount',
 'give',
 'me',
 'call',
 'used',
 'hereby',
 'last',
 'towards',
 'though',
 'sixty',
 'top',
 'per',
 'never',
 'move',
 'than',
 'almost',
 'made',
 'and',
 'did',
 'whole',
 'else',
 'our',
 'he',
 'anything',
 'for',
 'back',
 'although',
 'only',
 'meanwhile',
 'say',
 'am',
 'some',
 'might',
 'between',
 'twenty',
 'quite',
 'using',
 'hers',
 'a

##### Getting Lemma and Stop words

In [710]:
docx = nlp("This is how John Walker was walking. He was also running beside the lawn.")

In [711]:
# Lemmatizing of tokens
for word in docx:
    print(word.text,"Lemma =>",word.lemma_)

This Lemma => this
is Lemma => be
how Lemma => how
John Lemma => John
Walker Lemma => Walker
was Lemma => be
walking Lemma => walk
. Lemma => .
He Lemma => he
was Lemma => be
also Lemma => also
running Lemma => run
beside Lemma => beside
the Lemma => the
lawn Lemma => lawn
. Lemma => .


In [712]:
# Lemma that are not pronouns
for word in docx:
    if word.lemma_ != "-PRON-":
        print(word.lemma_.lower().strip())

this
be
how
john
walker
be
walk
.
he
be
also
run
beside
the
lawn
.


In [713]:
# List Comprehensions of our Lemma
[word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in docx]

['this',
 'be',
 'how',
 'john',
 'walker',
 'be',
 'walk',
 '.',
 'he',
 'be',
 'also',
 'run',
 'beside',
 'the',
 'lawn',
 '.']

In [714]:
# Filtering out Stopwords and Punctuations
for word in docx:
    if word.is_stop == False and not word.is_punct:
#     if word.is_stop != True and not word.is_punct:
        print(word)

John
Walker
walking
running
lawn


In [715]:
# Stop words and Punctuation In List Comprehension
[ word for word in docx if word.is_stop == False and not word.is_punct ]

[John, Walker, walking, running, lawn]

In [716]:
# Use the punctuations of string module
import string
punctuations = string.punctuation

In [717]:
# Creating a Spacy Parser
from spacy.lang.en import English
parser = English()

In [718]:
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    return mytokens

#### Machine Learning With SKlearn

In [719]:
# ML Packages
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

In [720]:
#Custom transformer using spaCy 
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    def fit(self, X, y=None, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}

# Basic function to clean the text 
def clean_text(text):     
    return text.strip().lower()

In [721]:
# Vectorization
vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1)) 
classifier = LinearSVC(dual=False)

In [722]:
# Using Tfidf
tfvectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [723]:
# Splitting Data Set
from sklearn.model_selection import train_test_split

In [724]:
# Features and Labels
X = df['Message']
ylabels = df['Target']

In [725]:
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.1, random_state=42)

In [726]:
# Create the  pipeline to clean, tokenize, vectorize, and classify 
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', classifier)])

In [727]:
X_train.to_csv("test.csv")

In [729]:
# Fit our data
pipe.fit(X_train,y_train)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
# Predicting with a test dataset
sample_prediction = pipe.predict(X_test)

In [ ]:
# Prediction Results
# 1 = Positive review
# 0 = Negative review
for (sample,pred) in zip(X_test,sample_prediction):
    print(sample,"Prediction=>",pred)

In [ ]:
# Accuracy
print("Accuracy: ",pipe.score(X_test,y_test))
print("Accuracy: ",pipe.score(X_test,sample_prediction))

In [ ]:
# Accuracy
print("Accuracy: ",pipe.score(X_train,y_train))

In [ ]:
# Another random review
pipe.predict(["This was a great movie"])

In [ ]:
example = ["I do enjoy my job",
 "What a poor product!,I will have to get a new one",
 "I feel amazing!"]

In [ ]:
pipe.predict(example)

In [ ]:
#### Using Tfid

In [ ]:
# Create the  pipeline to clean, tokenize, vectorize, and classify 
pipe_tfid = Pipeline([("cleaner", predictors()),
                 ('vectorizer', tfvectorizer),
                 ('classifier', classifier)])

In [ ]:
pipe_tfid.fit(X_train,y_train)

In [ ]:
sample_prediction1 = pipe_tfid.predict(X_test)

In [ ]:
for (sample,pred) in zip(X_test,sample_prediction1):
    print(sample,"Prediction=>", pred)

In [ ]:
print("Accuracy: ",pipe_tfid.score(X_test,y_test))
print("Accuracy: ",pipe_tfid.score(X_test,sample_prediction1))

In [ ]:
### Jesse JCharis
### J-Secur1ty
### Jesus Saves @ JCharisTech